<a href="https://colab.research.google.com/github/itsmeayushipanday/Chatbot-Using-Python-And-Machine-Learning/blob/main/Chatbot_Using_Python_And_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DATA PREPARATION**

IMPORTING LIBRARIES

In [51]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

READ JSON FILE AND PROCESS THE REQUIRED FIELDS

In [52]:
with open('intents.json') as file:
    data = json.load(file)

In [53]:
#The labels list stores unique intent tags,
#The training_sentences and training_labels lists are used for training the model.
#The responses list stores possible responses for each intent.
training_sentences = []
training_labels = []
labels = []
responses = []

In [54]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

LABEL ENCODER METHOD

In [55]:
#Label encoding initialization (to convert categorical labels into numerical labels)
lbl_encoder = LabelEncoder()

In [56]:
#Fitting the encoder (to fit the encoder on the training_labels)
lbl_encoder.fit(training_labels)

LabelEncoder()

In [57]:
#Transforming labels (converts the original string-based labels in the training_labels list to corresponding numerical labels )
training_labels = lbl_encoder.transform(training_labels)

TOKENIZATION (Used in natural language processing to split paragraphs and sentences into smaller units that can be more easily assigned meaning)

In [58]:
# Setting parameters
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

In [59]:
#Tokenizer initialization and fitting
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)

In [60]:
#Word index (unique integer index assigned to each word based on its frequency in the training data)
word_index = tokenizer.word_index

In [61]:
#Converting Sentences to Sequences
sequences = tokenizer.texts_to_sequences(training_sentences)

In [62]:
#Pdding sequences (to ensure that all sequences have the same length by padding them with zeros)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

TRAINING A NEURAL NETWORK

In [63]:
print(training_labels[:33])
print(padded_sequences[:34])

[4 4 4 4 4 3 3 3 7 7 7 7 0 0 0 6 6 6 5 5 5 5 5 5 5 2 2 2 2 2 1 1 1]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 29]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 30]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10 31 18]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 32]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 33]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 34]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 35  2 36]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 37]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 19]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 38  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 39 40]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 19 11 41  8]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 20 12  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  9 12  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [64]:
print("padded_sequences shape:", padded_sequences.shape)
print("training_labels shape:", training_labels.shape)

padded_sequences shape: (33, 20)
training_labels shape: (33,)


In [65]:
from keras.api._v2.keras import activations
#Creating the sequential model
model = Sequential()

#Adding layers to model
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D()) #performs global average pooling over the temporal dimension of the sequence.
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))

In [66]:
#Output layer
model.add(Dense(num_classes, activation='softmax'))

In [67]:
#Compiling the Model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_2   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 dense_8 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [68]:
print("Length of padded_sequences:", len(padded_sequences))
print("Length of training_labels:", len(training_labels))

Length of padded_sequences: 33
Length of training_labels: 33


In [69]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 12ms/step - loss: 2.0795 - accuracy: 0.1515
Epoch 2/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0783 - accuracy: 0.1515
Epoch 3/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0778 - accuracy: 0.1515
Epoch 4/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0771 - accuracy: 0.1515
Epoch 5/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0763 - accuracy: 0.1515
Epoch 6/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0757 - accuracy: 0.1515
Epoch 7/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0752 - accuracy: 0.1515
Epoch 8/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0747 - accuracy: 0.1515
Epoch 9/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0742 - accuracy: 0.1515
Epoch 10/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0738 - accuracy: 0.1515
Epoch 11/500
2/2 [=========

SAVING THE MODEL FOR FUTURE USE

In [70]:
#To save the trained model
model.save("chat_model")

import pickle
#To save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

#To save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

READY TO BUILD A CHATBOT WITH PYTHON AND OUR TRAINED MACHINE LEARNING MODEL

In [77]:
#Import Libraries:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder


!pip install --upgrade colorama
import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

In [ ]:
y

In [78]:
#Load Intent Data
with open("intents.json") as file:
  data = json.load(file)

In [79]:
#Define the chat Function
def chat():

  # Load trained model
  model = keras.models.load_model('chat_model')

  # Load tokenizer object
  with open('tokenizer.pickle', 'rb') as handle:
    tockenizer = pickle.load(handle)

  # Load label encoder object
  with open('label_encoder.pickle', 'rb') as enc:
    lbl_encoder = pickle.load(enc)

  #parameters
  max_len = 20

  while True:
    print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
    inp = input()
    if inp.lower() == "quit":
      break

    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    for i in data['intents']:
      if i['tag'] == tag:
        print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))


print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Hi
1/1 [==============================] - 0s 129ms/step
ChatBot: Hi
User: who are you
1/1 [==============================] - 0s 28ms/step
ChatBot: I.m Joana, your bot assistant
User: what is your name
1/1 [==============================] - 0s 24ms/step
ChatBot: I.m Joana, your bot assistant
User: i need your help
1/1 [==============================] - 0s 25ms/step
ChatBot: Tell me how can assist you
User: i want to create an account
1/1 [==============================] - 0s 34ms/step
ChatBot: You can just easily create a new account from our web site
User: i have a complaint
1/1 [==============================] - 0s 29ms/step
ChatBot: Please mention your complaint, we will reach you and sorry for any inconvenience caused
User: you are so humble
1/1 [==============================] - 0s 32ms/step
ChatBot: My pleasure
User: happy to contact you
1/1 [==============================] - 0s 28ms/step
ChatBot: Yes Sure, How can I support 